<a href="https://colab.research.google.com/github/xanasa14/MLImplementations/blob/master/Sentences_SimilaritiesEspanol.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Installing neeeded software

In [174]:


!pip install --upgrade gensim
!pip install -U pip setuptools wheel
#pip install -U spacy

!python -m spacy download es_core_news_sm
!spacy download es_core_news_sm
#!python -m spacy download es_core_news_md
#!spacy download es_core_news_md
from gensim.models import Word2Vec
from gensim.models.keyedvectors import KeyedVectors
import spacy
print(spacy.__version__)

nlp = spacy.load('es_core_news_sm')
 


  Using cached gensim-3.8.3-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
  Using cached gensim-3.8.2-cp36-cp36m-manylinux1_x86_64.whl (24.2 MB)
  Using cached pip-21.0-py3-none-any.whl (1.5 MB)
  Using cached pip-20.3.4-py2.py3-none-any.whl (1.5 MB)
  Using cached setuptools-52.0.0-py3-none-any.whl (784 kB)
  Using cached setuptools-51.3.3-py3-none-any.whl (786 kB)
  Using cached wheel-0.36.2-py2.py3-none-any.whl (35 kB)
  Using cached wheel-0.36.1-py2.py3-none-any.whl (34 kB)
     |████████████████████████████████| 16.2 MB 1.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
     |████████████████████████████████| 16.2 MB 1.1 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('es_core_news_sm')
2.2.4


Loading the model from gensim's vectors, and 
Creating my sentences to be compared


In [118]:

modelo = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/GensimSpanish/sbw_vectors.bin", binary=True)


news_headlines= []


news_headline0 = "Te gustan las empanadas?"
news_headline1 = "Te gustaron las empanadas?"
news_headline2 = "Sopa de pollo es lo mejor que hay"
news_headline3 = "habéis comido las empanadas?"
news_headline4 = "habéis degustado de las empanadas?"
news_headline5 = "Tengo que estudiar mucho para el examen que me agrada"

news_headlines_withoutLemma = [news_headline0 ,news_headline1,news_headline2, news_headline3,news_headline4,news_headline5]
#https://unipython.com/curso-de-procesamiento-de-textos-gensim/

Text Processing

Before finding out, we need to do some text process to be consistent with every word; Consquently, we will get rid of punctuations and we will also apply lemmatization using the Spanish version of Spacy


In [119]:
# Python program to remove punctuation from a given string 
# Function to remove punctuation 
def Punctuation(string): 
    string= string.replace("-PRON-","")      
  
    # punctuation marks 
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~-'''
  
    # traverse the given string and if any punctuation 
    # marks occur replace it with null 
    for x in string.lower(): 
        if x in punctuations: 
            string = string.replace(x, "") 
  
    # Print string without punctuation 
    return (string) 
#Lemmatizing from beginning 
for headlines in news_headlines_withoutLemma:
   doc = nlp(Punctuation(headlines))
   line = " ".join([token.lemma_ for token in doc])
   line= line.replace("-PRON-","") 
   line= line.replace("  "," ")      
     

   print(line)
   news_headlines.append(line)
print(news_headlines)

Te gustar los empanar
Te gustar los empanar
Sopa de pollo ser el mejor que haber
haber comer los empanar
haber degustar de los empanar
Tengo que estudiar mucho parir el examen que me agradar
['Te gustar los empanar', 'Te gustar los empanar', 'Sopa de pollo ser el mejor que haber', 'haber comer los empanar', 'haber degustar de los empanar', 'Tengo que estudiar mucho parir el examen que me agradar']


Remove Stopwords:

Some words are not as impactful in a sentece as others. We call this removing stopwords from our sentences



In [120]:
headline_tokens = []
for news_headline in news_headlines:
    headline_tokens.append([token.text.lower() for token in nlp(news_headline) if not token.is_stop])

print(headline_tokens)

[['gustar', 'empanar'], ['gustar', 'empanar'], ['sopa', 'pollo'], ['comer', 'empanar'], ['degustar', 'empanar'], ['estudiar', 'parir', 'examen', 'agradar']]


We will apply gensim to see how distant are these sentences, or bag of words


In [75]:
subject_headline = news_headlines[0]
subject_token = headline_tokens[0]

print('Headline: ', subject_headline)
print('=' * 50)
print()
lineNumber = 0
for token, headline in zip(headline_tokens, news_headlines):
    print('-' * 50)
    print(token)
    print('Comparing to:', news_headlines_withoutLemma[lineNumber])
    lineNumber += 1
    distance = modelo.wmdistance(subject_token, token)
    print('distance = %.4f' % distance)

Headline:  Te gustar los empanar

--------------------------------------------------
['gustar', 'empanar']
Comparing to: Te gustan las empanadas?
distance = 0.0000
--------------------------------------------------
['gustar', 'empanar']
Comparing to: Te gustaron las empanadas?
distance = 0.0000
--------------------------------------------------
['sopa', 'pollo']
Comparing to: Sopa de pollo es lo mejor que hay
distance = 4.0874
--------------------------------------------------
['comer', 'empanar']
Comparing to: habéis comido las empanadas?
distance = 1.8441
--------------------------------------------------
['degustar', 'empanar']
Comparing to: habéis degustado de las empanadas?
distance = 2.3674
--------------------------------------------------
['estudiar', 'parir', 'examen']
Comparing to: Tengo que estudiar mucho para el examen
distance = 4.1222


Creating a function that does the gensim similarity and also using it to find some close word to it, word by word. I also use lemmatization on the close words 


In [172]:
#points
print(news_headline0)
print(news_headline4)
print(headline_tokens[0])
print(headline_tokens[4])
def get_Gensim_Sim(sentence1, sentence2):
  points = 0 
  for word0 in sentence1:
    for word1 in sentence2:
      if (word0 == word1):
        points +=1
        break 
      else:
        word = word0
        sim = modelo.most_similar(positive=[word1],topn=30)
        lemmatized = "" 
        for i in range(len(sim)):
          doc = nlp(sim[i][0])
          lemmatized = " ".join([token.lemma_ for token in doc])
          #print(lemmatized, sim[i][0])
          if (word == lemmatized):
            
            print(word, lemmatized)
            print("it is here")
            points += 0.7
            break
           


  score = points/len(headline_tokens[0])
  print(str(score * 100) + "% similar")


get_Gensim_Sim(headline_tokens[0],headline_tokens[4])



Te gustan las empanadas?
habéis degustado de las empanadas?
['gustar', 'empanar']
['degustar', 'empanar']
empanar empanar
it is here
85.0% similar
